# cjio API tutorial

In this tutorial we explore what is possible with `cjio`'s API. I refer to `cjio`'s command line interface as *CLI*.

The CLI is what you use when you invoke `cjio` from the command line, such as:
```
$ cjio some_city_model.json validate
```

The API is what you use from `cjio` when working with a city model in a Python script.

In [1]:
import os
from cjio import cityjson

Set up the paths for the tutorial.

In [14]:
package_dir = os.path.dirname('.')
schema_dir = os.path.join(package_dir, 'cjio', 'schemas', '1.0.0')
data_dir = os.path.join(package_dir, 'example_data')

## Load the city model

We are working with a subset of the city model of Rotterdam. This file is included with `cjio` as an example data set. The `load()` method loads a CityJSON file into a CityJSON object.

In [16]:
p_subset = os.path.join(data_dir, 'rotterdam', 'rotterdam_subset.json')
cm_r = cityjson.load(p_subset)
print(type(cm_r))

<class 'cjio.cityjson.CityJSON'>


## Using the CLI commands in the API
You can use any of the CLI commands on a CityJSON object. However, not all CLI commands are mapped 1-to-1 to CityJSON methods, plus you need to provide the parameters yourself, as for example in case of the `validate()` method. The reason for this is that the CLI commands are wrapper functions that take care of passing parameters and parsing outputs. And we haven't harmonized the CLI and the API yet. 

In [17]:
cm_r.validate(folder_schemas=schema_dir)

-- Validating the syntax of the file
	(using the schemas cjio/schemas/1.0.0/cityjson.schema.json)
-- Validating the internal consistency of the file (see docs for list)
	--Vertex indices coherent
	--Specific for CityGroups
	--Semantic arrays coherent with geometry
	--Root properties
	--Empty geometries
	--Duplicate vertices
	--Orphan vertices
	--CityGML attributes


(True,
 False,
 [],
 ["WARNING: attributes 'TerrainHeight' not in CityGML schema",
  '\t(16 CityObjects have this warning)',
  "WARNING: attributes 'bron_tex' not in CityGML schema",
  '\t(16 CityObjects have this warning)',
  "WARNING: attributes 'voll_tex' not in CityGML schema",
  '\t(16 CityObjects have this warning)',
  "WARNING: attributes 'bron_geo' not in CityGML schema",
  '\t(16 CityObjects have this warning)',
  "WARNING: attributes 'status' not in CityGML schema",
  '\t(16 CityObjects have this warning)'])

## Explore the city model

We can print the basic information about the city model. Note that `print()` returns the same information as the `info` command in the CLI.

In [10]:
print(cm_r)

{
  "cityjson_version": "1.0",
  "epsg": 28992,
  "cityobjects_total": 16,
  "cityobjects_present": [
    "Building"
  ],
  "vertices_total": 383,
  "transform/compressed": true,
  "geom_primitives_present": [
    "MultiSurface"
  ],
  "materials": false,
  "textures": true
}


## Process the objects in the model

## Save or Export

